### Het maken en Trainen van een YOLO Model

De notebook is bedoeld om een dataset voor objectdetectie voor te bereiden, te trainen met behulp van YOLO (You Only Look Once), en vervolgens een voorspelling te maken met een getraind model. Het is onderverdeeld in verschillende secties: het voorbereiden van annotaties, het splitsen van de dataset, het trainen van het model en het maken van voorspellingen.

Dit script maakt gebruik van de volgende modules:

1. **`os`**: Wordt gebruikt voor het werken met bestands- en mapstructuren, zoals het aanmaken van mappen, controleren of bestanden bestaan, enzovoort.
2. **`PIL` (Python Imaging Library)**: Wordt gebruikt voor het openen, bewerken en opslaan van afbeeldingen.
3. **`random`**: Wordt gebruikt om willekeurige keuzes te maken, zoals het selecteren van bestanden of het randomiseren van gegevens.
4. **`shutil`**: Wordt gebruikt voor het kopiëren, verplaatsen of verwijderen van bestanden en mappen.
5. **`matplotlib.pyplot`**: Wordt gebruikt voor het visualiseren van data, zoals het tonen van afbeeldingen of grafieken.
6. **`cv2` (OpenCV)**: Wordt gebruikt voor computer vision-taken zoals het lezen, bewerken en verwerken van afbeeldingen.
7. **`ultralytics`**: Wordt gebruikt om het YOLO model te kunnen gebruiken.



In [62]:
import os
from PIL import Image
import random
import shutil 
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO

Dit script voert de volgende taken uit:

1. **Bestandspaden instellen**:
   - `coords_file`: Pad naar het bestand met coördinaten (`coords-idc.txt`).
   - `images_dir`: Pad naar de map met afbeeldingen (`complete_images`).
   - `output_dir`: Pad naar de map waarin de uitvoerbestanden (`annotations_yolo`) worden opgeslagen.

2. **Map aanmaken**:
   - Het script maakt de map `annotations_yolo` aan als deze nog niet bestaat (`os.makedirs(output_dir, exist_ok=True)`).



In [49]:

coords_file = "/students/2023-2024/Thema07/rumen_reactor/test/coords-idc.txt"
images_dir = "/students/2023-2024/Thema07/rumen_reactor/test/complete_images" 
output_dir = "annotations_yolo"  


os.makedirs(output_dir, exist_ok=True)


**Functie normalize_coords**:
   - Deze functie normaliseert de coördinaten van een rechthoek door de volgende berekeningen te maken:
     - **x_center** en **y_center**: Het centrum van de rechthoek ten opzichte van de breedte en hoogte van de afbeelding.
     - **width** en **height**: De breedte en hoogte van de rechthoek genormaliseerd naar de afbeelding.
   - De normalisatie wordt uitgevoerd door de originele coördinaten (minimaal en maximaal) te delen door de afmetingen van de afbeelding.

De normalize_coords functie wordt gebruikt om de coördinaten van objecten in afbeeldingen te transformeren naar een formaat dat geschikt is voor machine learning taken, zoals YOLO annotaties.

In [50]:
def normalize_coords(x_min, y_min, x_max, y_max, img_width, img_height):
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

Dit onderstaande script leest een bestand met coördinaten (`coords_file`) en verwerkt elke regel door de coördinaten van objecten in afbeeldingen om te zetten naar een formaat geschikt voor YOLO (You Only Look Once) objectdetectie. Voor elke regel worden de coördinaten gesplitst en omgezet naar gehele getallen (`x_min`, `y_min`, `x_max`, `y_max`). Het script opent vervolgens de bijbehorende afbeelding en haalt de afmetingen op. De coördinaten worden genormaliseerd door de functie `normalize_coords`, die de x- en y-positie, breedte en hoogte berekent ten opzichte van de afbeelding. De genormaliseerde waarden worden vervolgens opgeslagen in een uitvoerbestand (`output_file`) in het juiste YOLO-formaat. Het resultaat is een set van annotatiebestanden die kunnen worden gebruikt voor objectdetectie-modellen.


In [53]:
with open(coords_file, "r") as file:
    for line in file:
        parts = line.strip().split(",")
        image_id = parts[0]
        x_min, y_min, x_max, y_max = map(int, parts[1:])
       
        image_path = os.path.join(images_dir, f"{image_id}.jpeg")
        with Image.open(image_path) as img:
            img_width, img_height = img.size
       
        x_center, y_center, width, height = normalize_coords(
            x_min, y_min, x_max, y_max, img_width, img_height
        )
       

        output_file = os.path.join(output_dir, f"{image_id}.txt")
        with open(output_file, "a") as out:
            out.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")



Dit script maakt mappen aan voor het organiseren van afbeeldingen en labels in een dataset. Het stelt de volgende padvariabelen in:
- `labels_dir`: De map met de YOLO-annotaties.
- `output_dir`: De map waar de uiteindelijke dataset wordt opgeslagen.

Vervolgens worden submappen gedefinieerd voor de training en validatie van afbeeldingen en labels:
- `train_images_dir`: De map voor afbeeldingen in de trainingsset.
- `val_images_dir`: De map voor afbeeldingen in de validatieset.
- `train_labels_dir`: De map voor labelbestanden in de trainingsset.
- `val_labels_dir`: De map voor labelbestanden in de validatieset.

Het script maakt deze mappen aan, als ze nog niet bestaan, met `os.makedirs()` zodat de structuur van de dataset correct wordt voorbereid voor verder gebruik, zoals het trainen van een machine learning-model.


In [54]:
labels_dir = "annotations_yolo" 
output_dir = "dataset"  

train_images_dir = os.path.join(output_dir, "images/train")
val_images_dir = os.path.join(output_dir, "images/val")
train_labels_dir = os.path.join(output_dir, "labels/train")
val_labels_dir = os.path.join(output_dir, "labels/val")


os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)




Dit script verdeelt een lijst van afbeeldingsbestanden in een trainings- en validatieset. Het stelt eerst de verhoudingen in voor de verdeling van de data:
- `train_ratio`: Het percentage van de afbeeldingen dat naar de trainingsset gaat (80%).
- `val_ratio`: Het percentage van de afbeeldingen dat naar de validatieset gaat (20%).

Vervolgens worden alle afbeeldingsbestanden met de extensie `.jpeg` opgehaald uit de map `images_dir`. De lijst van afbeeldingsbestanden wordt willekeurig door elkaar geschud met `random.shuffle()` om de verdeling eerlijk te maken.

Het aantal afbeeldingen voor de trainingsset wordt berekend op basis van de `train_ratio`, en de lijst van afbeeldingsbestanden wordt opgesplitst in twee delen:
- `train_files`: De eerste 80% van de afbeeldingen (trainingsset).
- `val_files`: De resterende 20% van de afbeeldingen (validatieset).

Dit proces zorgt ervoor dat de afbeeldingen op een willekeurige manier worden verdeeld over de trainings- en validatiesets.


In [55]:

train_ratio = 0.8  
val_ratio = 0.2  


image_files = [f for f in os.listdir(images_dir) if f.endswith(".jpeg")]
random.shuffle(image_files) 


num_train = int(len(image_files) * train_ratio)
train_files = image_files[:num_train]
val_files = image_files[num_train:]


Dit script definieert een functie `move_files` die afbeeldings- en labelbestanden verplaatst van een bronmap naar een doelmap. De functie neemt de volgende argumenten:
- `image_list`: Een lijst van afbeeldingsbestanden die verplaatst moeten worden.
- `source_img_dir`: De map waar de bronafbeeldingen zich bevinden.
- `source_lbl_dir`: De map waar de bronlabelbestanden zich bevinden.
- `target_img_dir`: De map waar de afbeeldingsbestanden naartoe verplaatst moeten worden.
- `target_lbl_dir`: De map waar de labelbestanden naartoe verplaatst moeten worden.

De functie doorloopt elke afbeelding in `image_list`, genereert het bijbehorende labelbestand (met dezelfde naam, maar met de extensie `.txt`), en bouwt de volledige padnamen voor de bron- en doelbestanden. Als zowel het afbeeldingsbestand als het labelbestand bestaan in de bronmappen, worden ze verplaatst naar de doelmappen met behulp van `shutil.move()`. Als een van de bestanden ontbreekt, wordt er een waarschuwing weergegeven.

De functie wordt vervolgens aangeroepen om de afbeeldingen en labels voor zowel de trainings- als validatiesets van de bronmappen naar de doelmappen te verplaatsen:
- `move_files(train_files, images_dir, labels_dir, train_images_dir, train_labels_dir)`
- `move_files(val_files, images_dir, labels_dir, val_images_dir, val_labels_dir)`

Dit zorgt ervoor dat de afbeeldingen en bijbehorende labelbestanden correct worden verplaatst naar de juiste submappen voor de trainings- en validatiesets.


In [56]:
import os

def move_files(image_list, source_img_dir, source_lbl_dir, target_img_dir, target_lbl_dir):
    for img_file in image_list:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        src_img = os.path.join(source_img_dir, img_file)
        src_lbl = os.path.join(source_lbl_dir, label_file)
        tgt_img = os.path.join(target_img_dir, img_file)
        tgt_lbl = os.path.join(target_lbl_dir, label_file)

        if os.path.exists(src_img) and os.path.exists(src_lbl):
            shutil.move(src_img, tgt_img)
            shutil.move(src_lbl, tgt_lbl)
        else:
            print(f"Waarschuwing: Bestand {img_file} of {label_file} ontbreekt, overslaan.")


move_files(train_files, images_dir, labels_dir, train_images_dir, train_labels_dir)
move_files(val_files, images_dir, labels_dir, val_images_dir, val_labels_dir)


Dit script genereert een YAML-configuratiebestand voor de dataset. Het bevat de paden naar de trainings- en validatieafbeeldingen, het aantal klassen (`nc: 1`), en de lijst van klasse-namen (`names: ['tumor']`). Het bestand wordt opgeslagen als `dataset.yaml` in de `output_dir`. Het script geeft een bevestiging wanneer de dataset is gestructureerd en het bestand is opgeslagen.


In [57]:

yaml_content = f"""
train: {os.path.abspath(train_images_dir)}
val: {os.path.abspath(val_images_dir)}

nc: 1  # aantal klassen
names: ['tumor']  # lijst van klasse-namen
"""

yaml_path = os.path.join(output_dir, "dataset.yaml")
with open(yaml_path, "w") as yaml_file:
    yaml_file.write(yaml_content)

print(f"Dataset is gestructureerd in '{output_dir}' en configuratiebestand is opgeslagen als '{yaml_path}'.")


Dataset is gestructureerd in 'dataset' en configuratiebestand is opgeslagen als 'dataset/dataset.yaml'.


Hieronder wordt het model getraind hoe meer epochs hoe langer het duurt maar hoe beter het model.

In [81]:
model = YOLO("yolov8n.pt")

model.train(
    data = "dataset/dataset.yaml",
    epochs = 20,
    single_cls = True
)

Ultralytics 8.3.51 🚀 Python-3.11.2 torch-2.5.1+cu124 CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /homes/jrgommers/year 3/Deel_D/dataset/labels/train.cache... 223 images, 0 backgrounds, 2 corrupt: 100%|██████████| 223/223 [00:00<?, ?it/s]

train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10253.jpeg: 4 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10254.jpeg: 4 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10256.jpeg: 4 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10257.jpeg: 8 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10259.jpeg: 2 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10260.jpeg: 6 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10261.jpeg: 2 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10262.jpeg: 8 duplicate labels removed
train: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/train/10264.jpeg: 4 duplicate labels removed
train: WARNING ⚠️ /


val: Scanning /homes/jrgommers/year 3/Deel_D/dataset/labels/val.cache... 56 images, 0 backgrounds, 1 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]

val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10255.jpeg: 4 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10258.jpeg: 6 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10269.jpeg: 12 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10278.jpeg: 8 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10285.jpeg: 4 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10292.jpeg: 6 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10299.jpeg: 6 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10303.jpeg: 2 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset/images/val/10307.jpeg: 6 duplicate labels removed
val: WARNING ⚠️ /homes/jrgommers/year 3/Deel_D/dataset

...

                   all         55        153      0.386      0.301      0.267     0.0998



20 epochs completed in 0.529 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.11.2 torch-2.5.1+cu124 CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.41s/it]


                   all         55        153      0.361      0.281      0.263      0.101
Speed: 1.4ms preprocess, 96.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fe85d9eb950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Samenvatting van het trainingsproces:

### Resultaten:
- In de laatste epoch (epoch 20) zijn de resultaten als volgt:
  - **mAP50**: 0.361
  - **mAP50-95**: 0.0998
- Dit betekent dat het model nu in staat is om objecten in de afbeeldingen met een redelijke precisie te detecteren.
- De eindresultaten van de training worden opgeslagen in de map `runs/detect/train`.

Na de training wordt het model gevalideerd met de best presterende gewichten (`best.pt`), en de prestaties worden opnieuw gemeten. Het model is nu klaar voor gebruik in objectdetectie-taken.

De **`DetMetrics`** object van Ultralytics, dat gebruikt wordt om de prestaties van een objectdetectiemodel te meten. Het bevat:

- **`ap_class_index`**: Geeft aan welke klasse wordt geëvalueerd (bijv. alleen de "tumor"-klasse).
- **`box`**: Bevat gegevens over hoe goed het model de objecten (zoals een tumor) heeft gelokaliseerd met de bounding boxes.
- **`confusion_matrix`**: Laat zien hoeveel fouten het model maakt bij het classificeren van objecten.
- **`curves`**: Lijst van grafieken (zoals Precision-Recall) die de prestaties van het model bij verschillende drempels laten zien.
- **`curves_results`**: Bevat de gegevens van die grafieken om te analyseren hoe goed het model presteert bij verschillende confidence-drempels.






Hieronder wordt het model gebruikt om een willekeurige JPEG-afbeelding, `8959.jpeg`, te analyseren. Verander de tekst naar het pad van jouw afbeelding om zelf een afbeelding te testen. refresh vervolgens je bestanden en in de map `runs/detect` verschijnt een nieuwe map genaamd `predict` met de resultaten van je test.


In [91]:
import matplotlib.pyplot as plt
import cv2

model = YOLO("/homes/jrgommers/year 3/Deel_D/runs/detect/train/weights/best.pt")


test_image_path = "8959.jpeg"
results = model.predict(source=test_image_path, save=True)



image 1/1 /homes/jrgommers/year 3/Deel_D/8959.jpeg: 640x576 2 tumors, 138.0ms
Speed: 6.8ms preprocess, 138.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs/detect/predict


Hieronder wordt de afbeelding ook nog eens weergegeven met behulp van `matplotlib` en `cv2`, en opgeslagen als een uitvoerbestand `output_image.jpg` in dezelfde map.


In [92]:


image = cv2.imread(test_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
   
for result in results:
    for box in result.boxes:
        xmin, ymin, xmax, ymax = map(int, box.xyxy[0].tolist())
        confidence = box.conf[0]
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        cv2.putText(image, f"{confidence:.2f}", (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.show()


output_path = "output_image.jpg"  
cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  


<Figure size 1000x1000 with 1 Axes>

True

Op de GitHub zie je de resultaten van onze eigen test met het model in de `val batch` en `predict`. De `val batch` bevat de werkelijke resultaten, terwijl de `predict` de voorspellingen van het model zijn.
